# Tutorial 14: Unified Monitoring for Web Apps Using AI Agent Service

## Overview
When you have a web application that consumes AI Agent Service, you need to correlate logs from both sources to get a complete picture of your system's behavior. This tutorial shows how to:

- Query logs from both App Service and AI Agent Service
- Merge and correlate the data using correlation IDs
- Create unified dashboards for end-to-end monitoring
- Track requests from web app through to AI agent execution

## Prerequisites
- Azure App Service with Application Insights enabled
- Azure AI Foundry Agent Service with diagnostic settings configured
- Both services sending logs to the same Log Analytics workspace (recommended)
- Correlation ID implementation in your web app

## Architecture Overview
```
User Request → App Service → AI Agent Service → Response
     ↓              ↓                ↓            ↓
App Insights   AppServiceHTTPLogs  AzureDiagnostics  App Insights
     ↓              ↓                ↓            ↓
        Log Analytics Workspace (Unified View)
```

## 1. Setup and Configuration

In [ ]:
# Install required packages
!pip install azure-monitor-query azure-identity pandas plotly -q

In [ ]:
from azure.monitor.query import LogsQueryClient, LogsQueryStatus
from azure.identity import DefaultAzureCredential
from datetime import datetime, timedelta
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Configuration
WORKSPACE_ID = "6ca23ecb-629e-43eb-91ba-e7b2b02512e7"  # Your Log Analytics workspace ID
WORKSPACE_NAME = "aivdkx5j4gv2x56"  # Your workspace name

# Resource information
APP_SERVICE_NAME = "your-app-service-name"  # Replace with your App Service name
AI_AGENT_SERVICE_NAME = "your-ai-agent-service"  # Replace with your AI Agent Service name

# Time range for queries
LOOKBACK_HOURS = 24

# Initialize the client
credential = DefaultAzureCredential()
logs_client = LogsQueryClient(credential)

## 2. Understanding the Data Sources

### App Service Logs Tables:
- **AppServiceHTTPLogs**: HTTP request logs from your web app
- **AppServiceConsoleLogs**: Console/stdout logs from your app
- **AppServiceAppLogs**: Application-level logs
- **AppTraces**: Application Insights traces
- **AppRequests**: Application Insights requests

### AI Agent Service Logs Tables:
- **AzureDiagnostics**: Agent service operations and diagnostics
- Contains: agent creation, thread operations, run execution, tool calls

### Key Correlation Fields:
- **CorrelationId / OperationId**: Links related operations across services
- **TimeGenerated**: Timestamp for temporal correlation
- **CallerIPAddress**: Can help identify request sources

## 3. Basic Log Queries

Let's start with basic queries to understand each service's logs.

In [ ]:
# Query 1: App Service HTTP Requests
app_service_query = f"""
AppServiceHTTPLogs
| where TimeGenerated > ago({LOOKBACK_HOURS}h)
| where CsHost contains '{APP_SERVICE_NAME}'
| project 
    TimeGenerated,
    Method = CsMethod,
    Path = CsUriStem,
    StatusCode = ScStatus,
    Duration = TimeTaken,
    ClientIP = CIp,
    UserAgent = CsUserAgent,
    Host = CsHost
| order by TimeGenerated desc
| take 10
"""

response = logs_client.query_workspace(WORKSPACE_ID, app_service_query, timespan=timedelta(hours=LOOKBACK_HOURS))

if response.status == LogsQueryStatus.SUCCESS:
    df_app = pd.DataFrame(response.tables[0].rows, columns=[col.name for col in response.tables[0].columns])
    print("\n📊 App Service HTTP Requests (Last 10):")
    print(df_app.to_string(index=False))
else:
    print(f"Query failed: {response.status}")

In [ ]:
# Query 2: AI Agent Service Operations
agent_service_query = f"""
AzureDiagnostics
| where TimeGenerated > ago({LOOKBACK_HOURS}h)
| where Resource contains '{AI_AGENT_SERVICE_NAME}' or ResourceProvider == 'MICROSOFT.COGNITIVESERVICES'
| project 
    TimeGenerated,
    Operation = OperationName,
    Status = ResultType,
    Duration = DurationMs,
    HTTPStatus = httpStatusCode_d,
    CallerIP = CallerIPAddress,
    Resource,
    Properties = properties_s
| order by TimeGenerated desc
| take 10
"""

response = logs_client.query_workspace(WORKSPACE_ID, agent_service_query, timespan=timedelta(hours=LOOKBACK_HOURS))

if response.status == LogsQueryStatus.SUCCESS:
    df_agent = pd.DataFrame(response.tables[0].rows, columns=[col.name for col in response.tables[0].columns])
    print("\n🤖 AI Agent Service Operations (Last 10):")
    print(df_agent.to_string(index=False))
else:
    print(f"Query failed: {response.status}")

## 4. Correlation Pattern 1: Time-Based Correlation

When you don't have explicit correlation IDs, you can correlate based on:
- Timestamps (requests happening within same time window)
- IP addresses (requests from same source)
- Request patterns

In [ ]:
# Time-based correlation query
time_correlation_query = f"""
let AppRequests = AppServiceHTTPLogs
    | where TimeGenerated > ago({LOOKBACK_HOURS}h)
    | where CsHost contains '{APP_SERVICE_NAME}'
    | project 
        AppTime = TimeGenerated,
        Method = CsMethod,
        Path = CsUriStem,
        AppStatus = ScStatus,
        AppDuration = TimeTaken,
        ClientIP = CIp;
let AgentOperations = AzureDiagnostics
    | where TimeGenerated > ago({LOOKBACK_HOURS}h)
    | where ResourceProvider == 'MICROSOFT.COGNITIVESERVICES'
    | project 
        AgentTime = TimeGenerated,
        Operation = OperationName,
        AgentStatus = ResultType,
        AgentDuration = DurationMs,
        AgentIP = CallerIPAddress;
AppRequests
| join kind=inner (
    AgentOperations
) on $left.ClientIP == $right.AgentIP
| where datetime_diff('second', AgentTime, AppTime) between (-5 .. 30)  // Agent call within 5s before to 30s after app request
| project 
    AppTime,
    Method,
    Path,
    AppStatus,
    AgentTime,
    Operation,
    AgentStatus,
    TimeDiff = datetime_diff('millisecond', AgentTime, AppTime),
    TotalDuration = AppDuration + AgentDuration,
    ClientIP
| order by AppTime desc
| take 20
"""

response = logs_client.query_workspace(WORKSPACE_ID, time_correlation_query, timespan=timedelta(hours=LOOKBACK_HOURS))

if response.status == LogsQueryStatus.SUCCESS:
    df_corr = pd.DataFrame(response.tables[0].rows, columns=[col.name for col in response.tables[0].columns])
    print("\n🔗 Time-Based Correlation (App Service ↔ AI Agent):")
    print(df_corr.to_string(index=False))
else:
    print(f"Query failed: {response.status}")

## 5. Correlation Pattern 2: Using Application Insights (Recommended)

If you're using Application Insights with both services, you can use `operation_Id` for precise correlation.

### Implementation in Your Web App:
```python
from opencensus.ext.azure.log_exporter import AzureLogHandler
from opencensus.trace import config_integration
import logging

# Configure Application Insights
logger = logging.getLogger(__name__)
logger.addHandler(AzureLogHandler(connection_string='YOUR_CONNECTION_STRING'))
config_integration.trace_integrations(['requests'])

# When calling AI Agent Service, pass the correlation ID
import uuid
correlation_id = str(uuid.uuid4())
logger.info(f"Calling AI Agent", extra={'custom_dimensions': {'correlation_id': correlation_id}})

# Pass correlation_id to AI Agent Service in headers
headers = {'X-Correlation-ID': correlation_id}
```

In [ ]:
# Query with Application Insights correlation
appinsights_correlation_query = f"""
let AppInsightsRequests = AppRequests
    | where TimeGenerated > ago({LOOKBACK_HOURS}h)
    | project 
        TimeGenerated,
        OperationId,
        RequestName = Name,
        AppDuration = DurationMs,
        AppSuccess = Success,
        ResultCode,
        URL = Url;
let AppInsightsTraces = AppTraces
    | where TimeGenerated > ago({LOOKBACK_HOURS}h)
    | where Message contains "AI Agent" or Message contains "correlation"
    | extend CorrelationId = tostring(customDimensions.correlation_id)
    | project TimeGenerated, OperationId, Message, CorrelationId;
let AgentOps = AzureDiagnostics
    | where TimeGenerated > ago({LOOKBACK_HOURS}h)
    | where ResourceProvider == 'MICROSOFT.COGNITIVESERVICES'
    | extend CorrelationId = tostring(properties_s)
    | project 
        AgentTime = TimeGenerated,
        OperationName,
        AgentStatus = ResultType,
        AgentDuration = DurationMs,
        CorrelationId;
AppInsightsRequests
| join kind=leftouter (AppInsightsTraces) on OperationId
| join kind=leftouter (AgentOps) on CorrelationId
| where isnotempty(CorrelationId)
| project 
    TimeGenerated,
    RequestName,
    OperationId,
    CorrelationId,
    AppDuration,
    AgentOperation = OperationName,
    AgentDuration,
    TotalDuration = AppDuration + AgentDuration,
    Success = case(AppSuccess == true and AgentStatus == "Success", "✅", "❌"),
    ResultCode
| order by TimeGenerated desc
| take 20
"""

response = logs_client.query_workspace(WORKSPACE_ID, appinsights_correlation_query, timespan=timedelta(hours=LOOKBACK_HOURS))

if response.status == LogsQueryStatus.SUCCESS:
    df_appinsights = pd.DataFrame(response.tables[0].rows, columns=[col.name for col in response.tables[0].columns])
    print("\n🎯 Application Insights Correlation (Precise):")
    print(df_appinsights.to_string(index=False))
else:
    print(f"Query failed: {response.status}")

## 6. End-to-End Request Journey

Track a complete request from web app through AI agent execution.

In [ ]:
# End-to-end request journey
e2e_journey_query = f"""
let TimeWindow = {LOOKBACK_HOURS}h;
let WebAppRequests = AppServiceHTTPLogs
    | where TimeGenerated > ago(TimeWindow)
    | where CsHost contains '{APP_SERVICE_NAME}'
    | extend RequestId = strcat(CIp, "_", format_datetime(TimeGenerated, 'yyyy-MM-dd-HH-mm-ss'))
    | project 
        RequestId,
        Stage = "1_WebRequest",
        TimeGenerated,
        Component = "App Service",
        Operation = strcat(CsMethod, " ", CsUriStem),
        Status = tostring(ScStatus),
        Duration = TimeTaken,
        Details = strcat("Client: ", CIp);
let AgentCalls = AzureDiagnostics
    | where TimeGenerated > ago(TimeWindow)
    | where ResourceProvider == 'MICROSOFT.COGNITIVESERVICES'
    | extend RequestId = strcat(CallerIPAddress, "_", format_datetime(TimeGenerated, 'yyyy-MM-dd-HH-mm-ss'))
    | project 
        RequestId,
        Stage = case(
            OperationName contains "Assistant", "2_AgentCreation",
            OperationName contains "Thread", "3_ThreadOperation",
            OperationName contains "Run", "4_RunExecution",
            OperationName contains "Tool", "5_ToolCall",
            "6_Other"
        ),
        TimeGenerated,
        Component = "AI Agent Service",
        Operation = OperationName,
        Status = ResultType,
        Duration = DurationMs,
        Details = strcat("Resource: ", Resource);
union WebAppRequests, AgentCalls
| order by RequestId, Stage, TimeGenerated
| take 50
"""

response = logs_client.query_workspace(WORKSPACE_ID, e2e_journey_query, timespan=timedelta(hours=LOOKBACK_HOURS))

if response.status == LogsQueryStatus.SUCCESS:
    df_journey = pd.DataFrame(response.tables[0].rows, columns=[col.name for col in response.tables[0].columns])
    print("\n🛤️ End-to-End Request Journey:")
    print(df_journey.to_string(index=False))
    
    # Group by RequestId to show complete journeys
    if not df_journey.empty:
        print("\n\n📋 Grouped by Request ID:")
        for request_id in df_journey['RequestId'].unique()[:3]:  # Show first 3 requests
            request_data = df_journey[df_journey['RequestId'] == request_id]
            print(f"\n{'='*80}")
            print(f"Request ID: {request_id}")
            print(f"{'='*80}")
            print(request_data[['Stage', 'Component', 'Operation', 'Status', 'Duration']].to_string(index=False))
else:
    print(f"Query failed: {response.status}")

## 7. Performance Analysis Across Services

In [ ]:
# Performance breakdown query
performance_analysis_query = f"""
let TimeWindow = {LOOKBACK_HOURS}h;
let AppServicePerf = AppServiceHTTPLogs
    | where TimeGenerated > ago(TimeWindow)
    | summarize 
        TotalRequests = count(),
        AvgDuration = avg(TimeTaken),
        P95Duration = percentile(TimeTaken, 95),
        SuccessRate = countif(ScStatus < 400) * 100.0 / count()
    by bin(TimeGenerated, 1h)
    | extend Service = "App Service";
let AgentServicePerf = AzureDiagnostics
    | where TimeGenerated > ago(TimeWindow)
    | where ResourceProvider == 'MICROSOFT.COGNITIVESERVICES'
    | summarize 
        TotalRequests = count(),
        AvgDuration = avg(DurationMs),
        P95Duration = percentile(DurationMs, 95),
        SuccessRate = countif(ResultType == "Success") * 100.0 / count()
    by bin(TimeGenerated, 1h)
    | extend Service = "AI Agent Service";
union AppServicePerf, AgentServicePerf
| order by TimeGenerated desc, Service
"""

response = logs_client.query_workspace(WORKSPACE_ID, performance_analysis_query, timespan=timedelta(hours=LOOKBACK_HOURS))

if response.status == LogsQueryStatus.SUCCESS:
    df_perf = pd.DataFrame(response.tables[0].rows, columns=[col.name for col in response.tables[0].columns])
    print("\n⚡ Performance Comparison:")
    print(df_perf.to_string(index=False))
    
    # Create visualization
    if not df_perf.empty:
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Request Volume', 'Average Duration', 'P95 Duration', 'Success Rate'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )
        
        for service in df_perf['Service'].unique():
            service_data = df_perf[df_perf['Service'] == service]
            
            # Request Volume
            fig.add_trace(
                go.Scatter(x=service_data['TimeGenerated'], y=service_data['TotalRequests'], 
                          name=f"{service} - Requests", mode='lines+markers'),
                row=1, col=1
            )
            
            # Average Duration
            fig.add_trace(
                go.Scatter(x=service_data['TimeGenerated'], y=service_data['AvgDuration'], 
                          name=f"{service} - Avg", mode='lines+markers'),
                row=1, col=2
            )
            
            # P95 Duration
            fig.add_trace(
                go.Scatter(x=service_data['TimeGenerated'], y=service_data['P95Duration'], 
                          name=f"{service} - P95", mode='lines+markers'),
                row=2, col=1
            )
            
            # Success Rate
            fig.add_trace(
                go.Scatter(x=service_data['TimeGenerated'], y=service_data['SuccessRate'], 
                          name=f"{service} - Success %", mode='lines+markers'),
                row=2, col=2
            )
        
        fig.update_layout(height=800, showlegend=True, title_text="Service Performance Comparison")
        fig.show()
else:
    print(f"Query failed: {response.status}")

## 8. Error Correlation and Root Cause Analysis

In [ ]:
# Error correlation query
error_correlation_query = f"""
let TimeWindow = {LOOKBACK_HOURS}h;
let AppErrors = AppServiceHTTPLogs
    | where TimeGenerated > ago(TimeWindow)
    | where ScStatus >= 400
    | extend ErrorWindow = bin(TimeGenerated, 1m)
    | project 
        ErrorWindow,
        AppErrorTime = TimeGenerated,
        AppErrorType = strcat("HTTP ", ScStatus),
        AppPath = CsUriStem,
        ClientIP = CIp;
let AgentErrors = AzureDiagnostics
    | where TimeGenerated > ago(TimeWindow)
    | where ResourceProvider == 'MICROSOFT.COGNITIVESERVICES'
    | where ResultType != "Success" or httpStatusCode_d >= 400
    | extend ErrorWindow = bin(TimeGenerated, 1m)
    | project 
        ErrorWindow,
        AgentErrorTime = TimeGenerated,
        AgentErrorType = ResultType,
        AgentOperation = OperationName,
        HTTPStatus = httpStatusCode_d,
        CallerIP = CallerIPAddress;
AppErrors
| join kind=inner (AgentErrors) on ErrorWindow
| where datetime_diff('second', AgentErrorTime, AppErrorTime) between (-10 .. 30)
| project 
    ErrorWindow,
    AppErrorTime,
    AppErrorType,
    AppPath,
    AgentErrorTime,
    AgentErrorType,
    AgentOperation,
    HTTPStatus,
    TimeDelta = datetime_diff('millisecond', AgentErrorTime, AppErrorTime),
    PossibleRootCause = case(
        HTTPStatus >= 500, "🔴 Agent Service Error",
        HTTPStatus >= 400 and HTTPStatus < 500, "🟡 Client/Request Error",
        "🔵 Other"
    )
| order by ErrorWindow desc
| take 20
"""

response = logs_client.query_workspace(WORKSPACE_ID, error_correlation_query, timespan=timedelta(hours=LOOKBACK_HOURS))

if response.status == LogsQueryStatus.SUCCESS:
    df_errors = pd.DataFrame(response.tables[0].rows, columns=[col.name for col in response.tables[0].columns])
    print("\n🔥 Correlated Errors (App Service ↔ AI Agent):")
    print(df_errors.to_string(index=False))
    
    if not df_errors.empty:
        print("\n\n📊 Error Distribution:")
        error_counts = df_errors['PossibleRootCause'].value_counts()
        print(error_counts)
else:
    print(f"Query failed: {response.status}")

## 9. Unified Monitoring Dashboard Query

This comprehensive query can be used in Azure Workbooks for unified monitoring.

In [ ]:
# Comprehensive unified monitoring query
unified_dashboard_query = f"""
let TimeRange = {LOOKBACK_HOURS}h;
// App Service Metrics
let AppMetrics = AppServiceHTTPLogs
    | where TimeGenerated > ago(TimeRange)
    | summarize 
        AppRequests = count(),
        AppSuccessRate = countif(ScStatus < 400) * 100.0 / count(),
        AppAvgDuration = avg(TimeTaken),
        AppP95Duration = percentile(TimeTaken, 95),
        App4xxErrors = countif(ScStatus >= 400 and ScStatus < 500),
        App5xxErrors = countif(ScStatus >= 500)
    by TimeWindow = bin(TimeGenerated, 5m);
// AI Agent Metrics
let AgentMetrics = AzureDiagnostics
    | where TimeGenerated > ago(TimeRange)
    | where ResourceProvider == 'MICROSOFT.COGNITIVESERVICES'
    | summarize 
        AgentOperations = count(),
        AgentSuccessRate = countif(ResultType == "Success") * 100.0 / count(),
        AgentAvgDuration = avg(DurationMs),
        AgentP95Duration = percentile(DurationMs, 95),
        AgentErrors = countif(ResultType != "Success")
    by TimeWindow = bin(TimeGenerated, 5m);
// Join metrics
AppMetrics
| join kind=fullouter (AgentMetrics) on TimeWindow
| project 
    TimeWindow = coalesce(TimeWindow, TimeWindow1),
    // App Service Metrics
    AppRequests = coalesce(AppRequests, 0),
    AppSuccessRate = round(coalesce(AppSuccessRate, 0), 2),
    AppAvgDuration = round(coalesce(AppAvgDuration, 0), 2),
    AppP95Duration = round(coalesce(AppP95Duration, 0), 2),
    AppErrors = coalesce(App4xxErrors, 0) + coalesce(App5xxErrors, 0),
    // AI Agent Metrics
    AgentOperations = coalesce(AgentOperations, 0),
    AgentSuccessRate = round(coalesce(AgentSuccessRate, 0), 2),
    AgentAvgDuration = round(coalesce(AgentAvgDuration, 0), 2),
    AgentP95Duration = round(coalesce(AgentP95Duration, 0), 2),
    AgentErrors = coalesce(AgentErrors, 0),
    // Combined Metrics
    TotalOperations = coalesce(AppRequests, 0) + coalesce(AgentOperations, 0),
    CombinedSuccessRate = case(
        (coalesce(AppRequests, 0) + coalesce(AgentOperations, 0)) > 0,
        round(((coalesce(AppSuccessRate, 0) * coalesce(AppRequests, 0)) + (coalesce(AgentSuccessRate, 0) * coalesce(AgentOperations, 0))) / (coalesce(AppRequests, 0) + coalesce(AgentOperations, 0)), 2),
        0.0
    ),
    TotalErrors = coalesce(App4xxErrors, 0) + coalesce(App5xxErrors, 0) + coalesce(AgentErrors, 0),
    HealthStatus = case(
        (coalesce(AppSuccessRate, 0) >= 99 and coalesce(AgentSuccessRate, 0) >= 99), "✅ Healthy",
        (coalesce(AppSuccessRate, 0) >= 95 and coalesce(AgentSuccessRate, 0) >= 95), "🟡 Degraded",
        "🔴 Unhealthy"
    )
| order by TimeWindow desc
"""

response = logs_client.query_workspace(WORKSPACE_ID, unified_dashboard_query, timespan=timedelta(hours=LOOKBACK_HOURS))

if response.status == LogsQueryStatus.SUCCESS:
    df_unified = pd.DataFrame(response.tables[0].rows, columns=[col.name for col in response.tables[0].columns])
    print("\n📊 Unified Service Monitoring Dashboard:")
    print(df_unified.to_string(index=False))
    
    # Summary statistics
    if not df_unified.empty:
        print("\n\n📈 Summary (Last 24h):")
        print(f"Total App Service Requests: {df_unified['AppRequests'].sum():.0f}")
        print(f"Total AI Agent Operations: {df_unified['AgentOperations'].sum():.0f}")
        print(f"Average App Success Rate: {df_unified['AppSuccessRate'].mean():.2f}%")
        print(f"Average Agent Success Rate: {df_unified['AgentSuccessRate'].mean():.2f}%")
        print(f"Total Errors: {df_unified['TotalErrors'].sum():.0f}")
        print(f"Current Health: {df_unified.iloc[0]['HealthStatus']}")
else:
    print(f"Query failed: {response.status}")

## 10. Cost Analysis Across Services

In [ ]:
# Cost estimation query
cost_analysis_query = f"""
let TimeWindow = {LOOKBACK_HOURS}h;
// App Service usage
let AppServiceUsage = AppServiceHTTPLogs
    | where TimeGenerated > ago(TimeWindow)
    | summarize 
        DailyRequests = count(),
        TotalDataTransferMB = sum(CsBytes + ScBytes) / 1024.0 / 1024.0,
        AvgComputeTime = avg(TimeTaken)
    by Day = bin(TimeGenerated, 1d)
    | extend Service = "App Service";
// AI Agent usage
let AgentServiceUsage = AzureDiagnostics
    | where TimeGenerated > ago(TimeWindow)
    | where ResourceProvider == 'MICROSOFT.COGNITIVESERVICES'
    | summarize 
        DailyOperations = count(),
        TotalComputeTimeMs = sum(DurationMs),
        AssistantCreations = countif(OperationName contains "Assistant"),
        ThreadOperations = countif(OperationName contains "Thread"),
        RunExecutions = countif(OperationName contains "Run"),
        ToolCalls = countif(OperationName contains "Tool")
    by Day = bin(TimeGenerated, 1d)
    | extend 
        Service = "AI Agent Service",
        // Estimated costs (update with your pricing)
        EstimatedCost = (AssistantCreations * 0.001) + (RunExecutions * 0.01) + (ToolCalls * 0.005);
union AppServiceUsage, AgentServiceUsage
| order by Day desc, Service
"""

response = logs_client.query_workspace(WORKSPACE_ID, cost_analysis_query, timespan=timedelta(hours=LOOKBACK_HOURS))

if response.status == LogsQueryStatus.SUCCESS:
    df_cost = pd.DataFrame(response.tables[0].rows, columns=[col.name for col in response.tables[0].columns])
    print("\n💰 Cost & Usage Analysis:")
    print(df_cost.to_string(index=False))
else:
    print(f"Query failed: {response.status}")

## 11. Creating an Azure Workbook with Unified Queries

Let's create a workbook JSON that you can import to Azure Portal.

In [ ]:
# Generate unified monitoring workbook
unified_workbook = {
    "version": "Notebook/1.0",
    "items": [
        {
            "type": 1,
            "content": {
                "json": "# 🔄 Unified Web App + AI Agent Monitoring\n\nMonitor your web application and AI Agent Service in a single view. Track request flows, correlate errors, and analyze end-to-end performance."
            }
        },
        {
            "type": 9,
            "content": {
                "version": "KqlParameterItem/1.0",
                "parameters": [
                    {
                        "id": "time-range-param",
                        "version": "KqlParameterItem/1.0",
                        "name": "TimeRange",
                        "type": 4,
                        "isRequired": True,
                        "value": {"durationMs": 86400000},
                        "typeSettings": {
                            "selectableValues": [
                                {"durationMs": 3600000, "createdTime": "Last hour"},
                                {"durationMs": 14400000, "createdTime": "Last 4 hours"},
                                {"durationMs": 43200000, "createdTime": "Last 12 hours"},
                                {"durationMs": 86400000, "createdTime": "Last 24 hours"},
                                {"durationMs": 259200000, "createdTime": "Last 3 days"},
                                {"durationMs": 604800000, "createdTime": "Last 7 days"}
                            ]
                        }
                    },
                    {
                        "id": "workspace-param",
                        "version": "KqlParameterItem/1.0",
                        "name": "Workspace",
                        "type": 1,
                        "isRequired": True,
                        "value": WORKSPACE_NAME,
                        "label": "Log Analytics Workspace"
                    }
                ]
            }
        },
        {
            "type": 1,
            "content": {
                "json": "## 📊 Service Health Overview"
            }
        },
        {
            "type": 3,
            "content": {
                "version": "KqlItem/1.0",
                "query": unified_dashboard_query,
                "size": 0,
                "title": "Unified Service Metrics",
                "timeContext": {"durationMs": 86400000},
                "timeContextFromParameter": "TimeRange",
                "queryType": 0,
                "resourceType": "microsoft.operationalinsights/workspaces",
                "visualization": "table",
                "gridSettings": {
                    "formatters": [
                        {"columnMatch": "AppSuccessRate", "formatter": 8, "formatOptions": {"palette": "greenRed"}},
                        {"columnMatch": "AgentSuccessRate", "formatter": 8, "formatOptions": {"palette": "greenRed"}},
                        {"columnMatch": "CombinedSuccessRate", "formatter": 8, "formatOptions": {"palette": "greenRed"}},
                        {"columnMatch": "TotalErrors", "formatter": 8, "formatOptions": {"palette": "red"}},
                        {"columnMatch": "HealthStatus", "formatter": 18, "formatOptions": {"thresholdsOptions": "icons", "thresholdsGrid": [
                            {"operator": "contains", "thresholdValue": "Healthy", "representation": "success"},
                            {"operator": "contains", "thresholdValue": "Degraded", "representation": "warning"},
                            {"operator": "Default", "thresholdValue": None, "representation": "error"}
                        ]}}
                    ]
                }
            }
        },
        {
            "type": 1,
            "content": {
                "json": "## 🔗 Request Correlation"
            }
        },
        {
            "type": 3,
            "content": {
                "version": "KqlItem/1.0",
                "query": time_correlation_query,
                "size": 0,
                "title": "Correlated App + Agent Requests",
                "timeContext": {"durationMs": 86400000},
                "timeContextFromParameter": "TimeRange",
                "queryType": 0,
                "resourceType": "microsoft.operationalinsights/workspaces",
                "visualization": "table"
            }
        },
        {
            "type": 1,
            "content": {
                "json": "## 🔥 Error Analysis"
            }
        },
        {
            "type": 3,
            "content": {
                "version": "KqlItem/1.0",
                "query": error_correlation_query,
                "size": 0,
                "title": "Correlated Errors & Root Cause",
                "timeContext": {"durationMs": 86400000},
                "timeContextFromParameter": "TimeRange",
                "queryType": 0,
                "resourceType": "microsoft.operationalinsights/workspaces",
                "visualization": "table"
            }
        }
    ],
    "styleSettings": {},
    "fromTemplateId": "sentinel-UserWorkbook",
    "$schema": "https://github.com/Microsoft/Application-Insights-Workbooks/blob/master/schema/workbook.json"
}

# Save the workbook
with open('unified_webapp_agent_workbook.json', 'w') as f:
    json.dump(unified_workbook, f, indent=2)

print("✅ Unified workbook created: unified_webapp_agent_workbook.json")
print("\n📋 Import Instructions:")
print("1. Go to Azure Portal → Monitor → Workbooks")
print("2. Click '+ New'")
print("3. Click '</>' (Advanced Editor)")
print("4. Paste the contents of unified_webapp_agent_workbook.json")
print("5. Click 'Apply' → Save the workbook")

## 12. Best Practices for Unified Monitoring

### 1. **Implement Correlation IDs**
```python
# In your web app
import uuid
from flask import request, g

@app.before_request
def before_request():
    g.correlation_id = request.headers.get('X-Correlation-ID', str(uuid.uuid4()))
    
# When calling AI Agent Service
headers = {
    'X-Correlation-ID': g.correlation_id,
    'Authorization': f'Bearer {token}'
}
response = requests.post(agent_url, headers=headers, json=data)
```

### 2. **Use the Same Log Analytics Workspace**
- Configure both App Service and AI Agent Service to send logs to the same workspace
- Simplifies cross-service queries
- Reduces data transfer costs

### 3. **Enable Application Insights**
- Use Application Insights SDK in your web app
- Automatic correlation via `operation_Id`
- Better dependency tracking

### 4. **Set Up Alerts**
- Alert on correlated failures (app error + agent error)
- Alert on latency spikes in either service
- Alert on correlation mismatches (requests with no agent calls)

### 5. **Monitor Key Metrics**
- **Request Flow**: % of app requests that trigger agent calls
- **End-to-End Latency**: Total time from app request to final response
- **Success Correlation**: Both services succeed or identify where failure occurs
- **Cost Efficiency**: Agent calls per app request ratio

### 6. **Retention and Sampling**
- Configure appropriate retention for both services
- Consider sampling for high-volume apps (keep 100% for errors)
- Archive old logs to cheaper storage if needed

## Summary

This tutorial covered:
- ✅ Querying logs from App Service and AI Agent Service
- ✅ Time-based and correlation ID-based merging
- ✅ End-to-end request journey tracking
- ✅ Performance analysis across services
- ✅ Error correlation and root cause analysis
- ✅ Cost analysis for both services
- ✅ Creating unified monitoring dashboards
- ✅ Best practices for implementation

### Next Steps:
1. Implement correlation IDs in your web app
2. Configure diagnostic settings for both services
3. Import the unified workbook to Azure Portal
4. Set up alerts based on the queries
5. Monitor and optimize based on insights

### Useful Links:
- [Application Insights Correlation](https://learn.microsoft.com/en-us/azure/azure-monitor/app/correlation)
- [KQL Reference](https://learn.microsoft.com/en-us/azure/data-explorer/kusto/query/)
- [Azure Workbooks Documentation](https://learn.microsoft.com/en-us/azure/azure-monitor/visualize/workbooks-overview)
- [App Service Diagnostic Logs](https://learn.microsoft.com/en-us/azure/app-service/troubleshoot-diagnostic-logs)